# Quickstart for data modeling and power-ops-sdk for cogshop runs
NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file)

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been set up in Cognite Data Fusion (CDF).

Configuration of the PowerOpsClient (and resync) is done through a yaml file and environment variables.


### Client Credentials

Create an `.env` file on the with the following format
  ```sh
    PROJECT=<cdf-project>
    TENANT_ID=<tenant-id>
    CLUSTER=<cdf-cluster>
    CLIENT_ID=<client-id>
    CLIENT_SECRET=<client-secret>
  ```
  

In [ ]:
# You can control which configuration files are used.
# In this case, the configuration files are located two levels above, in the root of the repository.
from dotenv import load_dotenv
from pathlib import Path
from cognite.powerops import PowerOpsClient
from cognite.powerops._version import __version__

# Adjust the path to the configs file if needed
root = Path().resolve().parent

load_dotenv(dotenv_path=str(root / ".env"))
powerops = PowerOpsClient.from_config(str(root /"power_ops_config.yaml"))

print(__version__)  # Print the version of the package
print(powerops.cdf.config)

0.114.1
{   'api_subversion': '20230101',
    'base_url': 'https://bluefield.cognitedata.com',
    'client_name': 'CognitePygen:1.2.21:SDK:power-ops-sdk',
    'credentials': <cognite.client.credentials.OAuthClientCredentials object at 0x1270cde50>,
    'file_transfer_timeout': 600,
    'headers': {},
    'max_workers': 5,
    'project': 'power-ops-staging',
    'timeout': 30}


## STEPS:
1. Upload a yaml file that can be used by Shop, and return the external id
2. Set up a `ShopCase`, using the SDK
3. Write the `ShopCase` instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that `ShopCase`
5. List the `ShopResult` referencing the `ShopCase` 


### Step 1: Upload a file 

In the case that the file already exists, this step can be skipped. 

We need the external id for constructing the `ShopCase`

In [9]:
from pathlib import Path



def upload_file(file_name: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id="example_stavanger_case_file",
        name=file_name,
        data_set_id=powerops.datasets.write_dataset_id,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


def get_dataset() -> str:
    """Create a container in CDF and return the external id of the container"""
    monitor_data_set = powerops.cdf.data_sets.retrieve(
        external_id="powerops:monitor"
    )
    for dm in powerops.cdf.data_modeling.data_models:
         print(f"Data model: {dm.name} ({dm.external_id})")
    return monitor_data_set.external_id

    """"
    res = powerops.cdf.data_modeling.containers.retrieve(('power_ops_core', 'DataSetConfiguration'))
    print("Monitor data set is ", monitor_data_set)
    print("Container is ", res)
    """


# NB! The case in this example is minimal and just for demo purposes.
my_file_reference = upload_file(file_name="a_example_stavanger_with_commands.yaml")
get_dataset()

Data model: All DayAhead (all_DayAhead)
Data model: all_PowerOps (all_PowerOps)
Data model: BenchmarkingDayAhead (compute_BenchmarkingDayAhead)
Data model: ShopBasedDayAheadBidProcess (compute_ShopBasedDayAhead)
Data model: TotalBidMatrixCalculation (compute_TotalBidMatrixCalculation)
Data model: WaterValueBasedDayAheadBidProcess (compute_WaterValueBasedDayAheadBid)
Data model: DayAheadConfiguration (config_DayAheadConfiguration)
Data model: ResyncConfiguration (config_ResyncConfiguration)
Data model: AFRRBid (frontend_AFRRBid)
Data model: PowerAsset (frontend_Asset)
Data model: DayAheadBid (frontend_DayAheadBid)
Data model: CogShop (product_CogShop)


'powerops:monitor'

### Step 2 Preparing a ShopCase

`prepare_shop_case` creates a `Write` object can be written to CDF. 

We use the generated SDK (`_generated`) as they are a direct reflection of the Data Models on CDF. Here the data classes end in `Write` since we are going to use the to send data to CDF. They all have a corresponding class used for retrieving data from CDF.

(There is a more instance efficient way to prepare shop cases, by reusing scenarios. See the Advanced Guide, Example 2)

In [3]:
import datetime

# Unique identifier for the case, if not provided, an external id will be generated
EXAMPLE_CASE_EXTERNAL_ID = "example_case_external_id"

#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
file_tuple = (my_file_reference, "stavanger_case_file", False, "")


my_shop_case = powerops.cogshop.prepare_shop_case(
    shop_file_list=[file_tuple],
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2024, 5, 31, 22),
    end_time=datetime.datetime(2024, 6, 2, 22),
    model_name="my_model",
    scenario_name="my_scenario",
    case_external_id=EXAMPLE_CASE_EXTERNAL_ID,  # can be auto generated if set to None/not provided
)
my_shop_case

,value
space,power_ops_instances
external_id,example_case_external_id
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2024-05-31 22:00:00
end_time,2024-06-02 22:00:00
status,default
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 3: Upload the ShopCase

* Note: The upload is an `upsert` -- so if there are overlaps, the existing properties will merge (e.g. a list will be extended rather than replaced).
* Note 2: Multiple resources are created, this is because a `ShopCase` depends on the existence of `ShopModel`, `ShopScenario`, ... as described in the beginning of this guide. 



In [4]:
powerops.v1.upsert(my_shop_case)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='example_case_external_id', version=1) at 0x1067d6a50>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:99b900ae65054f51b7d41ab2081d8583', version=1) at 0x11b5eeb50>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:8e27d47622a941e4b42be209a4879234', version=1) at 0x10c203690>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:ad6fe67294f7498aa12323e4c15b4cb3', version=1) at 0x10c0f8590>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='example_case_external_id:shopfile:ad6fe67294f7498aa12323e4c15b4cb3', version=1) at 0x11b417090>], time_series=[], files=[], sequences=[])

Notice that the retrieved case is not suffixed with `Write`. However, it is possible to convert it to a `Write` object by doing `.as_write()`, make changes and then upsert again.

In [5]:
from cognite.powerops.client._generated.v1.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(my_shop_case.external_id)
retrieved_shop_case

,value
space,power_ops_instances
external_id,example_case_external_id
data_record,"{'version': 1, 'last_updated_time': 2025-08-07..."
node_type,None
scenario,shopscenario:99b900ae65054f51b7d41ab2081d8583
start_time,2024-05-31 22:00:00+00:00
end_time,2024-06-02 22:00:00+00:00
status,default
shop_files,None


*It is known that the `retrive` endpoint can be unstable. We have a graphQL fallback for this case. See the advanced guide.*

### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [6]:
powerops.cogshop.trigger_shop_case(my_shop_case.external_id)

### Step 5: View the ShopResult generated 

It may take a while to run (Cog)Shop.

A new things to note:
* If the run is not completed then no results are returned. 
* If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.
* Technically it is possible to run `trigger_shop_case` multiple times for the same case. For this reason we suggest using lists to view `ShopResult`s. 
  

Explanations for (some) of the fields of the `ShopResult`: 

* External id: The external id of the `ShopResult`
* Case: The external id of the `ShopCase` that the instance belongs to
* Objective value: The objective value of the Shop execution
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains the keys `last_updated_time` and `created_time`. These can be used to differentiate results from the same `ShopCase`



#### 1. Using `list_shop_results_for_case`

Note that this return type is a list from `data_classes`, which means it can be easily converted to a pandas dataframe.

*While this `list` is not usally unstable, we still maintain a graphQL fallback. See the advanced guide.* 


In [7]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

result_list: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=my_shop_case.external_id
)  # This can easily be converted to a pandas DataFrame
result_list

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,data_record
0,power_ops_instances,shop_result__882de10d-cc87-4f84-9024-a2fcbc2e4e66,example_case_external_id,"{'total': -380036.7316799998, 'load_value': 0....",example_stavanger_case_file,POWEROPS_SHOP_post-run-882de10d-cc87-4f84-9024...,POWEROPS_SHOP_shop-882de10d-cc87-4f84-9024-a2f...,POWEROPS_SHOP_cplex-882de10d-cc87-4f84-9024-a2...,"{'version': 1, 'last_updated_time': 2025-01-30..."


#### 2. Using the external ID of a ShopResult to retrieve

If we know a specific `ShopResult` is of interest, we can use its external id to retrieve it and inspect it  

In [8]:
shop_result = powerops.cogshop.retrieve_shop_result(result_list[0].external_id)

In [9]:
shop_result

,value
space,power_ops_instances
external_id,shop_result__882de10d-cc87-4f84-9024-a2fcbc2e4e66
data_record,"{'version': 1, 'last_updated_time': 2025-01-30..."
node_type,None
case,example_case_external_id
objective_value,"{'total': -380036.7316799998, 'load_value': 0...."
pre_run,example_stavanger_case_file
post_run,POWEROPS_SHOP_post-run-882de10d-cc87-4f84-9024...
messages,POWEROPS_SHOP_shop-882de10d-cc87-4f84-9024-a2f...
cplex_logs,POWEROPS_SHOP_cplex-882de10d-cc87-4f84-9024-a2...


In [10]:
# Downloading one of the files to tmp directory and printing the first 10 lines
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    log_file_path = str(Path(tmp_dir) / "log_file.log")
    powerops.cdf.files.download_to_path(external_id=shop_result.cplex_logs, path=log_file_path)
    with open(log_file_path) as log_file:
        print("".join(log_file.readlines()[:10]))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Output_CloneLog                         -1
CPXPARAM_Simplex_Limits_Iterations               10000000
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
LP Presolve eliminated 144 rows and 192 columns.
Aggregator did 192 substitutions.
Reduced LP has 337 rows, 339 columns, and 963 nonzeros.
Presolve time = 0.00 sec. (0.74 ticks)
Initializing dual steep norms . . .



## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [11]:
all_external_ids = [
    my_shop_case.scenario.external_id,
    my_shop_case.scenario.model.external_id,
    my_shop_case.external_id,
    my_shop_case.shop_files[0].external_id,
    shop_result.external_id,
]
all_external_ids

['shopscenario:13013d1cfd914e70a197757a85f78d68',
 'shopmodel:40a74ef8c04d4f3e89307f1c30aa2734',
 'example_case_external_id',
 'shopfile:74992bbb319d4927be22add52554b249',
 'shop_result__882de10d-cc87-4f84-9024-a2fcbc2e4e66']

In [12]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shopscenario:13013d1cfd914e70a197757a85f78d68'), NodeId(space='power_ops_instances', external_id='shopmodel:40a74ef8c04d4f3e89307f1c30aa2734'), NodeId(space='power_ops_instances', external_id='example_case_external_id'), NodeId(space='power_ops_instances', external_id='shopfile:74992bbb319d4927be22add52554b249'), NodeId(space='power_ops_instances', external_id='shop_result__882de10d-cc87-4f84-9024-a2fcbc2e4e66')], edges=[])